In [57]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn import svm
import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from ACAI_CNN_MNIST import Encoder

In [90]:
model = Encoder().cpu()
model.load_state_dict(torch.load('mnist_encoder.pt'))
model.eval()

Encoder(
  (layer): Sequential(
    (0): Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): LeakyReLU(negative_slope=0.2)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2)
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): LeakyReLU(negative_slope=0.2)
    (8): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): LeakyReLU(negative_slope=0.2)
    (10): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (11): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2)
    (13): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): LeakyReLU(negative_slope=0.2)
    (15): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (16): Conv2d(4, 4, kernel_size=(3, 3), str

In [72]:
test_dataset = torchvision.datasets.MNIST('/home/volta/Projects/Bakalarka/data',
                                     train=False,
                                     transform=torchvision.transforms.Compose([
                                         torchvision.transforms.ToTensor(),
                                         torchvision.transforms.Resize(32),
                                         torchvision.transforms.Normalize((0.1307,), (0.3081,))
                                         ])
                                     )

In [73]:
train_dataset = torchvision.datasets.MNIST('/home/volta/Projects/Bakalarka/data',
                                     train=True,
                                     transform=torchvision.transforms.Compose([
                                         torchvision.transforms.ToTensor(),
                                         torchvision.transforms.Resize(32),
                                         torchvision.transforms.Normalize((0.1307,), (0.3081,))
                                         ])
                                     )

In [103]:
train_dataloader = DataLoader(train_dataset, batch_size=256,num_workers=12)
test_dataloader = DataLoader(test_dataset, batch_size=256,num_workers=12)

In [118]:
def get_latents(data_loader):
    latents = []
    classes = []
    with torch.no_grad():
        for batch in data_loader:
            latents.append(model(batch[0]).numpy())
            classes.append(batch[1].numpy())
            
    latents = np.concatenate(latents)
    shape = latents.shape
    lats = latents.reshape(-1,shape[1]*shape[2]*shape[3])

    return lats, np.concatenate(classes)

In [119]:
def get_class(dataset, class_label):
    mask = classes == class_label
    return lats[mask]

In [120]:
label = 0
X_train = get_class(get_latents(train_dataloader),label)
X_test, Y_test = get_latents(test_dataloader)

In [189]:
clf = svm.OneClassSVM(nu=0.5, kernel="rbf", gamma='scale')
clf.fit(X_train)

OneClassSVM()

In [190]:
predicted = clf.predict(X_test)

In [191]:
predictions = predicted == 1
ground_truth = Y_test == label

In [192]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ground_truth, predictions)
tn, fp, fn, tp = cm.ravel()

In [193]:
accuracy = (tp+tn)/sum([tn,fp,fn,tp])
accuracy 

0.9315

In [194]:
precision = tp / (tp + fp)
precision

0.7159590043923866